<a href="https://colab.research.google.com/github/LBncl/AmazonMLInterviewQuestion/blob/main/NAFLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAFLD PROJECT

## Imports

In [234]:
# Imports
import missingno as msno
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import imblearn
import collections
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
import datetime
from sklearn.model_selection import cross_val_score
from numpy.ma.core import mean
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from numpy import random

## Data Cleaning

In [2]:
# Read in data
df_raw = pd.read_excel('master_with_nordic_and_multiBM.xlsx')

# Check if dataFrame is empty
if df_raw.empty:
    print('DataFrame is empty!')

In [156]:
# Subset data frame
main_df = df_raw.filter(items=['CPH_EV_AGE_CALC', 'TBL.PATIENT.INFO..PI_BL_GENDER', 'CPH_EV_CI_BMI_CALC', 
                         'TBL.ALL.EVENTS..AE_SF_ALCO_XS', 'insulin_resistance', 'hypertensive', 'waist_to_hip_ratio',
                         'idf_metabolic_syndrome', 'eGFR', 'dyslipidaemia', 'fibroscan_stiffness_reliable',
                         'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_AST_iuL', 'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
                         'TBL.ALL.EVENTS..AE_BR_PLT_109L', 'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC', 'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_IGA',
                         'TBL.ALL.EVENTS..AE_CD_OSA', 'LIT_NB_CK18_M30', 'LIT_NB_CK18_M65', 'LIT_NB_PRO_C3', 'LIT_NB_PRO_C6',
                          'LIT_NB_ELF', 'FIB4', 'NFS', 'APRI', 'ADAPT', 'FIBC3', 'ABC3D', 'BARD', 'AST_ALT_Ratio', 'response_3b','CPH_EV_EVENT_TYPE'])

In [ ]:
main_df.describe()

In [ ]:
main_df.dtypes

In [ ]:
msno.bar(main_df)

In [157]:
# Remove duplicates
print(main_df.shape)
main_df = main_df.drop_duplicates()
# Reset the index and drop original index column 
main_df = main_df.reset_index(drop=True)
print(main_df.shape)

(14236, 37)
(13772, 37)


In [ ]:
# View effect of changes
main_df

In [158]:
# Convert required variables to boolean 
features_raw = main_df

# Loop to check for boolean columns
for column in features_raw.iloc[:, 0:36]:
  if features_raw[column].max() == 1 and features_raw[column].min() == 0:
    features_raw[column] = features_raw[column].astype(bool)

In [ ]:
features_raw.dtypes

In [ ]:
features_raw

In [159]:
# Filter to only select baseline event types
print(main_df.shape)
features_raw['CPH_EV_EVENT_TYPE'] = features_raw['CPH_EV_EVENT_TYPE'].astype(str)
features_raw = features_raw.loc[features_raw['CPH_EV_EVENT_TYPE'] == 'Baseline']
# Reset the index and drop original index column 
features_raw = features_raw.reset_index(drop=True)
print(features_raw.shape)

(13772, 37)
(8991, 37)


In [ ]:
# View effect of changes
features_raw

In [98]:
# Remove baseline column
features_raw = features_raw.iloc[:, 0:36]

In [160]:
# Remove rows where there is no response value and set value to Bool
print(features_raw.shape)
print(features_raw['response_3b'].isna().sum())
features_raw = features_raw[features_raw['response_3b'].notna()]
print(features_raw.shape)

(8991, 37)
0
(8991, 37)


In [100]:
features_baseline = features_raw

In [ ]:
features_baseline.plot(kind="box", subplots=True, layout=(9,4), figsize=(50,50))

In [ ]:
features_baseline.dtypes

In [161]:
cols_normalise = features_baseline.select_dtypes(include=[np.float64])

# Normalise Data
scale = StandardScaler()
features_baseline[cols_normalise.columns] = scale.fit_transform(cols_normalise)

In [162]:
# Reset the index and drop original index column 
features_baseline = features_baseline.reset_index(drop=True)

In [ ]:
# View dataframe to confirm all changes
features_baseline

## Data Imputation Strategies

In [13]:
# Single imputation with mean to replace not a number (NaNs)
def nan2mean(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [14]:
# Single imputation with median to replace not a number (NaNs)
def nan2median(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [15]:
# Single imputation with most_frequent to replace not a number (NaNs)
def nan2most_frequent(fdf):
    cols = list(fdf.columns)
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [16]:
# Multiple Imputation by chained equation linear regression
def nan2mice(fdf):
    cols = list(fdf.columns)
    lr = LinearRegression()
    imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=50, imputation_order='roman',random_state=0)
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

In [17]:
# Imputation by K neasrest neighbours
def nan2knn(fdf):
    cols = list(fdf.columns)
    imp = KNNImputer(n_neighbors=2, weights="distance")
    fdf=imp.fit_transform(fdf)
    fdf = pd.DataFrame(fdf, columns=cols)
    return fdf

## New Imputation Strategy

In [164]:
# Stub dataframe to store new baseline rows
stub_df = pd.DataFrame(columns=['CPH_EV_AGE_CALC', 'TBL.PATIENT.INFO..PI_BL_GENDER', 'CPH_EV_CI_BMI_CALC', 
                         'TBL.ALL.EVENTS..AE_SF_ALCO_XS', 'insulin_resistance', 'hypertensive', 'waist_to_hip_ratio',
                         'idf_metabolic_syndrome', 'eGFR', 'dyslipidaemia', 'fibroscan_stiffness_reliable',
                         'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_AST_iuL', 'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
                         'TBL.ALL.EVENTS..AE_BR_PLT_109L', 'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC', 'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_IGA',
                         'TBL.ALL.EVENTS..AE_CD_OSA', 'LIT_NB_CK18_M30', 'LIT_NB_CK18_M65', 'LIT_NB_PRO_C3', 'LIT_NB_PRO_C6',
                          'LIT_NB_ELF', 'FIB4', 'NFS', 'APRI', 'ADAPT', 'FIBC3', 'ABC3D', 'BARD', 'AST_ALT_Ratio', 'response_3b'])

In [182]:
# Subset data frame
main_df_new = df_raw.filter(items=['CPH_EV_AGE_CALC', 'TBL.PATIENT.INFO..PI_BL_GENDER', 'CPH_EV_CI_BMI_CALC', 
                         'TBL.ALL.EVENTS..AE_SF_ALCO_XS', 'insulin_resistance', 'hypertensive', 'waist_to_hip_ratio',
                         'idf_metabolic_syndrome', 'eGFR', 'dyslipidaemia', 'fibroscan_stiffness_reliable',
                         'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_AST_iuL', 'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
                         'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
                         'TBL.ALL.EVENTS..AE_BR_PLT_109L', 'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC', 'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
                         'TBL.ALL.EVENTS..AE_BR_IGA',
                         'TBL.ALL.EVENTS..AE_CD_OSA', 'LIT_NB_CK18_M30', 'LIT_NB_CK18_M65', 'LIT_NB_PRO_C3', 'LIT_NB_PRO_C6',
                          'LIT_NB_ELF', 'FIB4', 'NFS', 'APRI', 'ADAPT', 'FIBC3', 'ABC3D', 'BARD', 'AST_ALT_Ratio', 'response_3b','CPH_EV_EVENT_TYPE', 'SPIC',
                          'CPH_ADM_BIOPSY_EVENT_DATE'])

# Remove duplicates
print(main_df_new.shape)
main_df_new = main_df_new.drop_duplicates()
# Reset the index and drop original index column 
main_df_new = main_df_new.reset_index(drop=True)
print(main_df_new.shape)

(14236, 39)
(14084, 39)


In [183]:
# Convert to string for comparrison
main_df_new['CPH_EV_EVENT_TYPE'] = main_df_new['CPH_EV_EVENT_TYPE'].astype(str)

# Drop observations with no date
main_df_new = main_df_new[main_df_new['CPH_ADM_BIOPSY_EVENT_DATE'].notna()]
print(main_df_new.shape)
# Reset the index and drop original index column 
main_df_new = main_df_new.reset_index(drop=True)

(13586, 39)


In [ ]:
# Group data by paitnet ID
main_df_grouped = main_df_new.groupby('SPIC')
# Loop over the agregated data
for group_name, df_group in main_df_grouped:
  # Sort the group based upon date
  df_group = df_group.sort_values(by=['CPH_ADM_BIOPSY_EVENT_DATE'])
  # Select the baseline observations
  baseline_row = df_group.loc[df_group['CPH_EV_EVENT_TYPE'] == "Baseline"]
  # Remove duplicate rows
  baseline_row = baseline_row.drop_duplicates()
  # Reset the index and drop original index column 
  baseline_row = baseline_row.reset_index(drop=True)
  # If baseline row is non existant tell user
  if baseline_row.empty:
    print('No baseline for paitent: ' + df_group['SPIC'].item())
  # If only baseline add basleine row to dataframe  
  elif df_group.shape[0] <= 1:
    stub_df = pd.concat([stub_df, baseline_row.iloc[:, 0:36]], ignore_index=True)
  # Only select data to impute that has multiple observations
  elif df_group.shape[0] > 1 :
    # Selects date of baseline observation
    start_date = baseline_row['CPH_ADM_BIOPSY_EVENT_DATE']
    # Convert from series to datetime
    start_date = datetime.datetime(start_date.dt.year,start_date.dt.month, start_date.dt.day)
    # Calculate absoulte time differenance
    df_group['Time Difference'] = abs(df_group['CPH_ADM_BIOPSY_EVENT_DATE'] - start_date)
    # Remove all rows where time difference is to large to impute
    df_group = df_group.loc[df_group['Time Difference'] < datetime.timedelta(days=200)]
    # Check again to see if any data is imputeable
    if df_group.shape[0] > 1:
      # Select relevant data to impute
      baseline_x = baseline_row.iloc[:, 0:36]
      # for each column in the baseline row 
      for (columnName, columnData) in baseline_x.iteritems():
        # Select value of current observation
        val = columnData.item()
        # If value is NaN
        if np.isnan(val):
          # For all other observations of the same paitent
          for i in range(df_group.shape[0]-1):
            # Select next observation 
            next = df_group.iloc[i+1]
            new_val = next[columnName].item()
            # If next value is NaN move on
            columnData.item = new_val
  # Select new imputed basline
  baseline_row = df_group.loc[df_group['CPH_EV_EVENT_TYPE'] == "Baseline"]
  stub_df = pd.concat([stub_df, baseline_row.iloc[:, 0:36]], ignore_index=True)
  
# Reomve duplicates
stub_df = stub_df.drop_duplicates()

In [185]:
# Remove rows where there is no response value
print(stub_df.shape)
stub_df = stub_df[stub_df['response_3b'].notna()]
# Reset the index and drop original index column 
stub_df = stub_df.reset_index(drop=True)
print(stub_df.shape)

(8567, 36)
(6956, 36)


In [169]:
# Drop uneeded columns
features_baseline_new_imputed = stub_df.iloc[:, 0:36]

In [170]:
# Loop to check for boolean columns
for column in features_baseline_new_imputed.iloc[:, 0:36]:
  if features_baseline_new_imputed[column].max() == 1 and features_baseline_new_imputed[column].min() == 0:
    features_baseline_new_imputed[column] = features_baseline_new_imputed[column].astype(bool)

In [186]:
cols_normalise = features_baseline_new_imputed.select_dtypes(include=[np.float64])

# Normalise Data
scale = StandardScaler()
features_baseline_new_imputed[cols_normalise.columns] = scale.fit_transform(cols_normalise)

In [187]:
# Reset the index and drop original index column 
features_baseline_new_imputed = features_baseline_new_imputed.reset_index(drop=True)

In [ ]:
# View changes to dataframe
features_baseline_new_imputed

## Imputation Methods

In [ ]:
# Create new imputed datasets
featuresKNN = nan2knn(features_baseline.iloc[:, 0:35])
featuresMean = nan2mean(features_baseline.iloc[:, 0:35])
featuresMedian = nan2median(features_baseline.iloc[:, 0:35])
featuresMICE = nan2mice(features_baseline.iloc[:, 0:35])
featuresMICE2 = nan2mice(features_baseline_new_imputed.iloc[:, 0:35])
featuresMostFrq = nan2most_frequent(features_baseline.iloc[:, 0:35])
featuresMostFrq = featuresMostFrq.apply(pd.to_numeric)

# Create list of all imputation strategies
imputed_features = [featuresKNN,featuresMean,featuresMedian,featuresMICE,featuresMostFrq]
imputed_features_names = ['featuresKNN','featuresMean','featuresMedian','featuresMICE','featuresMostFrq']

In [190]:
# Add the response value to the new imputed dataset
for df in imputed_features:
  df['response_3b'] = features_baseline['response_3b']

In [ ]:
# View a dataframe to make sure the response has been added
featuresMICE

In [192]:
# Add the response to new method imputed data
featuresMICE2['response_3b'] = features_baseline_new_imputed['response_3b']

In [ ]:
# View a dataframe to make sure the response has been added
featuresMICE2

In [195]:
# Update list of all imputation strategies
imputed_features = [featuresKNN,featuresMean,featuresMedian,featuresMICE,featuresMostFrq, featuresMICE2]
imputed_features_names = ['featuresKNN','featuresMean','featuresMedian','featuresMICE','featuresMostFrq', 'featuresMICE2']

In [ ]:
# Plots to show how the inputed data denstiy compares to the raw data
column_name = "CPH_EV_AGE_CALC"

d = {'raw': features_baseline[column_name], 
     'KNN Imputation': featuresKNN[column_name],
     'Mean Imputation': featuresMean[column_name],
     'Median Imputation': featuresMedian[column_name],
     'MICE Imputation': featuresMICE[column_name],
     'MostFrq Imputation': featuresMostFrq[column_name],
     'MICE New Imputation': featuresMICE2[column_name]
     }
fdata = pd.DataFrame(data=d)
 
fig, axes = plt.subplots(1, 7)
fig.suptitle('Raw vs Imputed Desity Plots')
fdata['raw'].plot.density(ax=axes[0], figsize = (15, 7))
fdata['Mean Imputation'].plot.density(ax=axes[1], figsize = (15, 7), color='red')
fdata['Median Imputation'].plot.density(ax=axes[2], figsize = (15, 7), color='red')
fdata['MICE Imputation'].plot.density(ax=axes[3], figsize = (15, 7), color='red')
fdata['MostFrq Imputation'].plot.density(ax=axes[4], figsize = (15, 7), color='red')
fdata['KNN Imputation'].plot.density(ax=axes[5], figsize = (15, 7), color='red')
fdata['MICE New Imputation'].plot.density(ax=axes[6], figsize = (15, 7), color='red')

axes[0].set_title('Raw')
axes[1].set_title('Mean Imputed')
axes[2].set_title('Median Imputed')
axes[3].set_title('MICE Imputed')
axes[4].set_title('MostFrq Imputed')
axes[5].set_title('KNN Imputed')
axes[6].set_title('MICE New Imputation')

axes[0].set_xlabel(column_name)
axes[1].set_xlabel(column_name)
axes[2].set_xlabel(column_name)
axes[3].set_xlabel(column_name)
axes[4].set_xlabel(column_name)
axes[5].set_xlabel(column_name)
axes[6].set_xlabel(column_name)

## Outlier Identifcation

In [ ]:
for df in imputed_features:
  # split into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:35], df.iloc[: , -1], test_size=0.33, random_state=1)
  # summarize the shape of the training dataset
  print('Original data shape: ' + str(X_train.shape) +  str(y_train.shape))
  # identify outliers in the training dataset
  iso = IsolationForest(contamination=0.1)
  yhat = iso.fit_predict(X_train)
  # select all rows that are not outliers
  mask = yhat != -1
  X_train, y_train = X_train.iloc[mask, :], y_train.iloc[mask]
  # summarize the shape of the updated training dataset
  print('Outliers removed: ' + str(X_train.shape) +  str(y_train.shape))

## SMOTE

In [132]:
# check version number
print(imblearn.__version__)

0.8.1


In [ ]:
# summarize class distributions for each imputed data set
for df in imputed_features:
  counter = collections.Counter(df['response_3b'])
  y = df['response_3b']
  X = df.iloc[:, 0:35]
  print(counter)

In [ ]:
from numpy.core.multiarray import where
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	plt.scatter(X.iloc[row_ix, 0], X.iloc[row_ix, 1], label=str(label))
plt.legend()
plt.show()

## Logistic Regression

In [ ]:
# Logistic Regression model
# Implement the model
logreg = LogisticRegression()

# hyperparameters, penalty values chosen to work with all solvers
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=logreg, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)

# SMOTE
over = SMOTE(sampling_strategy=0.4)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for i in range(len(imputed_features)):
  df = imputed_features[i]
  X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:35], df['response_3b'], test_size=0.2, random_state=42)

  # identify outliers in the training dataset
  iso = IsolationForest(contamination=0.1)
  yhat = iso.fit_predict(X_train)
  # select all rows that are not outliers
  mask = yhat != -1
  X_train, y_train = X_train.iloc[mask, :], y_train.iloc[mask]
  
  X_train, y_train = pipeline.fit_resample(X_train, y_train)

  grid_result = grid_search.fit(X_train, y_train)
  grid_predict = grid_result.predict(X_test)

  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), imputed_features_names[i])
  print(confusion_matrix(y_test, grid_predict))
  print(classification_report(y_test, grid_predict))

## XGBoost 

### Base XGBoost model

In [ ]:
# Base XGBoost model

# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df_name = imputed_features_names[i]
  X = df.iloc[:, 0:35]
  y = df['response_3b']
  scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
  # summarize performance
  print('Mean ROC AUC: %.5f' % mean(scores))
  print('For: ' + df_name)

### XGBoost SMOTE

In [ ]:
# SMOTE
over = SMOTE(sampling_strategy=0.35)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df_name = imputed_features_names[i]
  X = df.iloc[:, 0:35]
  y = df['response_3b']
  X, y = pipeline.fit_resample(X, y)
  scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
  # summarize performance
  print('Mean ROC AUC: %.5f' % mean(scores))
  print('For: ' + df_name)

### Cost-Sensitive XGBoost

In [ ]:
# XGBoost documnetation suggests a good way to estimate scale_pos_weight = total_negative_examples / total_positive_examples
estimate = 1869 / 7122

# define model
model = XGBClassifier(scale_pos_weight=estimate)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df_name = imputed_features_names[i]
  X = df.iloc[:, 0:35]
  y = df['response_3b']
  scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
  # summarize performance
  print('Mean ROC AUC: %.5f' % mean(scores))
  print('For: ' + df_name)


In [ ]:
# Same model as abouve with grid search for best scale_pos_weight value
# define model
model = XGBClassifier()
# define grid
weights = []
for i in range(5):
  weights.append(random.uniform(estimate, estimate*10))
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# evaluate model
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df_name = imputed_features_names[i]
  X = df.iloc[:, 0:35]
  y = df['response_3b']
  # execute the grid search
  grid_result = grid.fit(X, y)
  # report the best configuration
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  print('For: ' + df_name)

### Final XGBoost Model

In [256]:
# SMOTE
over = SMOTE(sampling_strategy=0.35)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# define model
model = XGBClassifier(scale_pos_weight=estimate)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
for i in range(len(imputed_features)):
  df = imputed_features[i]
  df_name = imputed_features_names[i]
  X = df.iloc[:, 0:35]
  y = df['response_3b']
  X, y = pipeline.fit_resample(X, y)
  counter = collections.Counter(y)
  print(counter)
  scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
  # summarize performance
  print('Mean ROC AUC: %.5f' % mean(scores))
  print('For: ' + df_name)

Counter({False: 2492, True: 2492})
Mean ROC AUC: 0.73912
For: featuresKNN
Counter({False: 2492, True: 2492})
Mean ROC AUC: 0.76754
For: featuresMean
Counter({False: 2492, True: 2492})
Mean ROC AUC: 0.76527
For: featuresMedian
Counter({False: 2492, True: 2492})
Mean ROC AUC: 0.74623
For: featuresMICE
Counter({False: 2492, True: 2492})


KeyboardInterrupt: ignored